# HW 6 : Hand written digits recognition

## First step : loading and processing the data
Load the images into a numpy array

In [40]:
import numpy as np
import pandas as pd

In [5]:
# Creation of a dataframe to hold the images
data_set = pd.read_csv("image_0.txt", header=None, names=\
                      ["Pixel_"+str(x) for x in range(400)])
print(data_set.info())
data_set.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Columns: 400 entries, Pixel_0 to Pixel_399
dtypes: float64(395), int64(5)
memory usage: 15.3 MB
None


,Pixel_0,Pixel_1,Pixel_2,Pixel_3,Pixel_4,Pixel_5,Pixel_6,Pixel_7,Pixel_8,Pixel_9,...,Pixel_390,Pixel_391,Pixel_392,Pixel_393,Pixel_394,Pixel_395,Pixel_396,Pixel_397,Pixel_398,Pixel_399
0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


Now we add the labels to the data set

In [8]:
data_set["Label"] = pd.read_csv("label.txt", header=None, names=["Label"])
# Replace the 10s with actual zeros
def ten_to_zero(x):
    if x == 10:
        return 0
    return x
data_set["Label"] = data_set["Label"].apply(func=ten_to_zero)
# Reorganize the columns so that Label is first on the list
cols = data_set.columns.tolist()
cols = cols[-1:] + cols[:-1]

data_set = data_set[cols]

print(data_set.info())
data_set.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Columns: 401 entries, Label to Pixel_399
dtypes: float64(395), int64(6)
memory usage: 15.3 MB
None


,Label,Pixel_0,Pixel_1,Pixel_2,Pixel_3,Pixel_4,Pixel_5,Pixel_6,Pixel_7,Pixel_8,...,Pixel_390,Pixel_391,Pixel_392,Pixel_393,Pixel_394,Pixel_395,Pixel_396,Pixel_397,Pixel_398,Pixel_399
0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [10]:
def split_train_test(data, proportion=0.8):
    shuffled_data = data.sample(frac=1).reset_index(drop=True)
    
    train_data = shuffled_data[:int(0.8 * shuffled_data.shape[0])]\
            .reset_index(drop=True)
    test_data = shuffled_data[int(0.8 * shuffled_data.shape[0]):]\
            .reset_index(drop=True)
    
    return train_data, test_data

In [27]:
# Test the split function
train_data, test_data = split_train_test(data_set)
print(train_data.info())

   Label  Pixel_0  Pixel_1  Pixel_2  Pixel_3  Pixel_4  Pixel_5  Pixel_6  \
0      3        0        0      0.0      0.0      0.0      0.0      0.0   
1      4        0        0      0.0      0.0      0.0      0.0      0.0   
2      1        0        0      0.0      0.0      0.0      0.0      0.0   
3      8        0        0      0.0      0.0      0.0      0.0      0.0   
4      3        0        0      0.0      0.0      0.0      0.0      0.0   

   Pixel_7  Pixel_8    ...      Pixel_390  Pixel_391  Pixel_392  Pixel_393  \
0      0.0      0.0    ...            0.0        0.0        0.0        0.0   
1      0.0      0.0    ...            0.0        0.0        0.0        0.0   
2      0.0      0.0    ...            0.0        0.0        0.0        0.0   
3      0.0      0.0    ...            0.0        0.0        0.0        0.0   
4      0.0      0.0    ...            0.0        0.0        0.0        0.0   

   Pixel_394  Pixel_395  Pixel_396  Pixel_397  Pixel_398  Pixel_399  
0        0

## Now we define the parameters of our model
* We define a function to initialize parameteres randomly

In [97]:
def initialize_parameters(layers, n_inputs=400):
    """
        layers is a list. Each of its item represents one layer, and 
    the number it contains is the size of that layer. The last layer
    should always have the size of the expected output (in our case 10)
        Our model will then apply argmax to these 10 numbers to find out 
    which digit was detected.
    
        The return value is a dic containing a numpy array for each set
    of parameters
    """
    parameters = []
    
    slices = [n_inputs] + layers
    
    for i, layer in enumerate(layers):
        parameters.append(\
        np.random.rand((slices[i]+1), slices[i+1]))
        # We move the center of the parameters so that they are all
        # contained between -1 and 1
        parameters[i] = parameters[i] * 2 - 1
        
    return parameters

In [98]:
# Test the function 
params = initialize_parameters([25, 10])
for row in params:
    print(row.shape)
    
print(params[0][0, 0])

(401, 25)
(26, 10)
0.8644816097450265


## Implement the feed forward function
This step is just a couple of matrix multiplication, which are easy to implement and run very fast thanks to numpy.

To make it work, we must first define the sigmoid function that we will
be using for activation.

In [92]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

In [99]:
def feed_forward_batch(batch, params):
    """
        Given a batch of inputs, this function will return an equivalent
    batch of outputs. The intermediate values are also returned so they
    can be cached for back propagation.
    
    * Inputs : batch must be a numpy array of shape 400 * m where m
    represents the batch size.
    
    * Outputs : Has two return values : one is the output by the model,
    in the form of a vector of values, and the other is the list
    of intermediate outputs
    """
    # X is the matrix of values taht we will feed to the neural
    # network
    X = np.copy(batch)
    image_size = X.shape[0]
    
    ouptut = np.zeros((params[-1].shape[1]))
    cache = []
    
    
    for i, param_matrix in enumerate(params):
        cache_dic = {}
        # We pad the batch with a row of ones to create an intercept
        X = np.concatenate(np.ones(X.shape[0]), X, axis=0)
        # Then we multiply the padded batch of vectors with the params
        neuron_output = np.dot(X, param_matrix)
        # We cache the results for faster backprop
        cache_dic["pre_activation"] = neuron_output
        # Then we need to apply the activation function to the product
        activation = sigmoid(neuron_output)
        # We also store the results from after this operation because
        # at this point why not
        cache_dic["post_activation"] = activation
        
        cache.append(cache_dic)
        
    # Once this loop is completed, we should have our output ready
    return output, cache